In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd drive/MyDrive/Colab Notebooks/Thesis Project/

[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks/Thesis Project/'
/content


In [ ]:
!ls

drive  sample_data


In [ ]:
pip install transformers


In [ ]:
#import libraries
import numpy as np
import pandas as pd
import random
import re, string
import seaborn as sns
import matplotlib.pyplot as plt

#PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split


from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW


In [ ]:
# Load the dataset
df = pd.read_excel('/content/drive/MyDrive/nlp project dataset/BSMDD_v3_textcleaned - 21K.xlsx')
df.head()

,text,label
0,মানসিক শারীরিকভাবে অসুস্থ ক্লান্ত পুরো জীবন শা...,1
1,দয়া সাথে থাকুন অত্যন্ত দীর্ঘ আপনাকে পড়তে উত্...,1
2,জানতাম সাথে ভুল লোক খারাপ জীবন কাটিয়েছে সম্পূ...,1
3,অনেটিভ ইংরেজি স্পিকারের অনুসরণ বিরক্তিকর অপ্রত...,1
4,অনেটিভ ইংরেজি স্পিকারের অনুসরণ বিরক্তিকর অপ্রত...,1


In [ ]:
df.to_csv('preprocessed_dataset.csv', index=False)


In [ ]:
# Load preprocessed dataset
df = pd.read_csv('/content/preprocessed_dataset.csv')

# Split dataset into training and evaluation sets
train_df, eval_df = train_test_split(df, test_size=0.2, random_state=42)

# Load the BanglaBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained('sagorsarker/bangla-bert-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Set the maximum sequence length
max_seq_length = 512

# Tokenize and encode text data with max length
train_encodings = tokenizer(list(train_df['text']), truncation=True, padding='max_length', max_length=max_seq_length)
eval_encodings = tokenizer(list(eval_df['text']), truncation=True, padding='max_length', max_length=max_seq_length)

# Convert encodings to tensors
train_input_ids = torch.tensor(train_encodings['input_ids'])
train_attention_mask = torch.tensor(train_encodings['attention_mask'])
eval_input_ids = torch.tensor(eval_encodings['input_ids'])
eval_attention_mask = torch.tensor(eval_encodings['attention_mask'])

# Convert labels to tensors
train_labels = torch.tensor(list(train_df['label']))
eval_labels = torch.tensor(list(eval_df['label']))

# Create PyTorch datasets
train_dataset = torch.utils.data.TensorDataset(train_input_ids, train_attention_mask, train_labels)
eval_dataset = torch.utils.data.TensorDataset(eval_input_ids, eval_attention_mask, eval_labels)


In [ ]:
# Load the BanglaBERT model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained('sagorsarker/bangla-bert-base', num_labels=2)

# Set up the optimizer
optimizer = AdamW(model.parameters(), lr=1e-3)

# Set up the data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
eval_loader = torch.utils.data.DataLoader(eval_dataset, batch_size=8, shuffle=False)

# Fine-tuning BanglaBERT
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

model.safetensors:   0%|          | 0.00/660M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sagorsarker/bangla-bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(102025, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [ ]:
for epoch in range(3):  # Set the number of training epochs
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    model.eval()
    eval_accuracy = 0
    eval_loss = 0
    with torch.no_grad():
        for batch in eval_loader:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            logits = outputs.logits

            eval_loss += outputs.loss.item()
            eval_accuracy += (logits.argmax(dim=1) == labels).float().mean().item()

    eval_loss /= len(eval_loader)
    eval_accuracy /= len(eval_loader)
    print(f"Epoch {epoch+1}: Evaluation Loss: {eval_loss}, Evaluation Accuracy: {eval_accuracy}")


Epoch 1: Evaluation Loss: 0.7762936091553556, Evaluation Accuracy: 0.4990115572501273
